# Tutorial : Save, load and export models with keras


---

**Prerequisites:** 
* Python
* Tensorflow 2.0

**Concepts Covered:** 
* Deep Learning
* Keras




**Hours Needed to Complete:** 1.5 hours

# Background

----

**For Students:** In the tutorial, we are going to focus on learning to save, load and export models with keras and save model checkpoints during training. When finish the these tasks, you should be able to save, load and export keras in either keras or saved model format. These tasks are:

* Create Model

* Data preprocessing

* Model checkpoint during training

* Load Weights

* Saving complete model during training

* Load Models

* Saving weights and models

* Exporting and restroing saved model


## Tutorial Walkthrough
----


#### **Step 1**
Importing libraries and creating folders for future using, create a function to return model instances and take a look at the model architecture.

In [1]:
# Import neccessary package here.
import tensorflow as tf
import numpy as np
import os

# Create these four folders to save the model and weight in the future.
folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [2]:
#Create a function to create model here.
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


#### **Step 2**
Data preprocessing for keras MNMIST fashion dataset, normalizing and reshaping the examples. One-hot encoding the labels.

In [3]:
#Use variable dressing to save the imported data
dressing = tf.keras.datasets.fashion_mnist

#Extract the train and test images and their labels here.
(x_train, y_train), (x_test, y_test) = dressing.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


4423680/4422102 [==============================] - 0s 0us/step


#### **Step 3**
Model checkpoint during training: keras model checkpoint callback, some common model checkpoint callback arguments.

* **ModelCheckpoint callback is used in conjunction with training using model. fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.**

In [4]:
checkpoint_dir = 'weights/'
_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_accuracy:.4f}'),
            monitor='val_accuracy', save_weights_only=True, save_best_only=True
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 11ms/step - loss: 0.6762 - accuracy: 0.7705 - val_loss: 0.4890 - val_accuracy: 0.8301
Epoch 2/2
118/118 [==============================] - 1s 9ms/step - loss: 0.4383 - accuracy: 0.8476 - val_loss: 0.4487 - val_accuracy: 0.8449


In [5]:
os.listdir(checkpoint_dir)

['epoch_02_acc_0.8449.index',
 'epoch_02_acc_0.8449.data-00000-of-00001',
 'checkpoint',
 'epoch_01_acc_0.8301.index',
 'epoch_01_acc_0.8301.data-00000-of-00001']

* Explanation of arugments:


1.   monitor: quantity to monitor
2.   mode: we can choose max, min or auto to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For example: we need to maximize the val_acc and minimize the val_loss.
3. save_weights_only: only save the weight of the model or save the full model.
4. save_best_only: the latest best model according to the quantity monitored will not be overwritten.



#### **Step 4**
Load Weights, creating and evaluating a new model instance, loading a previously save weight from a checkpoint and evaluating the model again.

* When we save the weight of the model, it means we save all the paramteter from the model including trainable and non-trainable of the model which are in turn all the parameters used in the layers of the model.

In [6]:
# Initial a new model instance and evaluate it by using test data.
model = create_model()
print("test loss, test acc:", model.evaluate(x_test, y_test, verbose=False))

test loss, test acc: [2.3996856212615967, 0.057500001043081284]


In [52]:
# Use the model which saved before from the epoch2, see the difference test acc and loss.
model.load_weights('weights/epoch_02_acc_0.8803')
results = model.evaluate(x_test, y_test, verbose=False)
print("test loss, test acc:", results)

test loss, test acc: [0.32869505882263184, 0.880299985408783]


#### **Step 5**
Saving the complete model during training and save the model as h5 file format into models folder.

* When we set the parameter **'save_only_weights'=False**, we will save the whole model structure. We also need to change the saving format to .h5 as the model format.

In [10]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_accuracy:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 11ms/step - loss: 0.6939 - accuracy: 0.7703 - val_loss: 0.5175 - val_accuracy: 0.8219
Epoch 2/2
118/118 [==============================] - 1s 10ms/step - loss: 0.4381 - accuracy: 0.8457 - val_loss: 0.4775 - val_accuracy: 0.8241


In [11]:
# Now we have too models here.
os.listdir(models_dir)

['epoch_02_acc_0.8241.h5', 'epoch_01_acc_0.8219.h5']

#### **Step 6**
Load full models files from disk, we will again compare a new instance`s performance with a reload model from disk


In [12]:
# Initial a new model instance and evaluate it by using test data.
model = create_model()
print("test loss, test acc:", model.evaluate(x_test, y_test, verbose=False))

test loss, test acc: [2.4367785453796387, 0.07670000195503235]


In [13]:
model.load_weights('models/epoch_02_acc_0.8241.h5')
results = model.evaluate(x_test, y_test, verbose=False)
print("test loss, test acc:", results)

test loss, test acc: [0.47754356265068054, 0.8241000175476074]


#### **Step 7**
Manually Saving Weights and models, because we use model checkpoint before, we can manually save the weights and model from checkpoint.

In [17]:
model.save_weights('tmp/manually_saved_weights')
print(os.listdir('tmp'))
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['manually_saved_weights.index', 'manually_saved_weights.data-00000-of-00001', 'checkpoint', '.ipynb_checkpoints']
['manually_saved_weights.index', 'manually_saved_weights.data-00000-of-00001', 'manually_saved_model.h5', 'checkpoint', '.ipynb_checkpoints']


#### **Step 8**
Exporting and restoring savedModel Format, using the save weights and the save functions to export to and import from the saved model format.

In [18]:
model.save('model_name')
print(os.listdir('model_name'))
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_name/assets
['saved_model.pb', 'assets', 'variables']
[0.47754356265068054, 0.8241000175476074]
